# Part 1: Introduction and Getting Hands on The Data

This part aims to explore trends in recent data NYC 311 Service Request calls.

### Data Sources

- The 311 Service Requests (2010 to Present) from NYC Open Data
This dataset contains information about the time, location, complaint type, and status of more than 24 millions 311 service requests made in New York City within the past decade.

- NYC Department of City Planning's Community District Profiles
This dataset contains development and population information for each Community District in New York City. Community Board names, which correspond to community districts, can also be found in the 311 dataset.

## Obtaining the 311 Data
In this part, we are getting our hands on the data first to see the sense of how big the data is and which part that we want to focus on.

In [1]:
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# Load a required package 
# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 

import matplotlib.pyplot as plt
import seaborn as sns # visualization styling package

# A function that renders the figure in a notebook 
%matplotlib inline 

In [2]:
# For this part, I use the csv format to load the data
API = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv?'

In [3]:
# Selected data from May 20, 2021 to June 30, 2021
query = {'$select': '*',
         '$where': 'created_date>="2021-05-20" and created_date<"2021-10-12"', 
         '$limit': 1000000}

df = pd.read_csv(API + urlencode(query), 
                 parse_dates = ['created_date'],
                 dtype = {'incident_zip':'str'})
df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,50615719,2021-05-20,2021-05-20T06:19:04.000,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11432,85-10 KINGSTON PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.714722,-73.792810,"\n, \n(40.71472222322351, -73.79281026895278)"
1,50617501,2021-05-20,2021-05-27T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Food Cart Vendor,11103,BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.758975,-73.919000,"\n, \n(40.758974640414664, -73.91899955159224)"
2,50619934,2021-05-20,2021-05-20T14:13:30.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,11419,115-18 95 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.691489,-73.830291,"\n, \n(40.691488711180696, -73.83029110907381)"
3,50620098,2021-05-20,2021-05-27T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Food Cart Vendor,11375,107-26 71 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.720760,-73.844402,"\n, \n(40.72075965187039, -73.8444015846602)"
4,50621455,2021-05-20,2021-05-27T12:40:26.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Other (Explain Below),10306,2758 HYLAN BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.564639,-74.115623,"\n, \n(40.56463935497431, -74.1156225488924)"


In [4]:
print (len(df))
print (df['created_date'].min())
print (df['created_date'].max())

1000000
2021-05-20 00:00:00
2021-09-18 19:45:16


In [49]:
print (df.columns)

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location'],
      dtype='object')


## Cleaning the Data
In this part of cleaning the data, we want to remove unnecessary components/ columns and eliminate data with null values.

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830907 entries, 0 to 830906
Data columns (total 41 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   unique_key                      830907 non-null  int64         
 1   created_date                    830907 non-null  datetime64[ns]
 2   closed_date                     765155 non-null  object        
 3   agency                          830907 non-null  object        
 4   agency_name                     830907 non-null  object        
 5   complaint_type                  830907 non-null  object        
 6   descriptor                      793787 non-null  object        
 7   location_type                   707589 non-null  object        
 8   incident_zip                    805082 non-null  object        
 9   incident_address                786294 non-null  object        
 10  street_name                     786235 non-null  object 

In [51]:
#eliminate rows without lat/long and location type data
df = df[df.latitude.notnull()]
df = df[df.longitude.notnull()]
df = df[df.descriptor.notnull()]

In [52]:
#exclude unnecessary columns
df = df[['created_date', 'complaint_type', 'agency', 'community_board', 
         'latitude', 'longitude', 'descriptor', 'borough']]
print(df.columns)

Index(['created_date', 'complaint_type', 'agency', 'community_board',
       'latitude', 'longitude', 'descriptor', 'borough'],
      dtype='object')


### Descriptions of The Selected Columns

The descriptions for every column are available on the dataset's NYCOpenData profile. For this Assignment's Analysis, the descriptions for the columns included are as follows:

- Created Date: Date service request(SR) was created
- Complaint Type: This is the first level of a hierarchy identifying the topic of the incident or condition. Complaint Type may have a corresponding Descriptor (below) or may stand alone.
- Agency: Acronym of responding City Government Agency
- Community Board: Provided by geovalidation.
- Latitude: Geo based Lat of the incident location
- Longitude: eo based Long of the incident location
- Descriptor: This is associated to the Complaint Type, and provides further detail on the incident or condition. Descriptor values are dependent on the Complaint Type, and are not always required in SR.
- Borough: Provided by the submitter and confirmed by geovalidation.

### Reformatting Values for Consistency

In [53]:
# Here, we want to convert all of the values in the column to title case
df.complaint_type.unique()

array(['Water System', 'Illegal Parking', 'Noise - Street/Sidewalk',
       'Street Condition', 'Street Light Condition',
       'Noise - Residential', 'Noise - Commercial', 'New Tree Request',
       'Blocked Driveway', 'HEAT/HOT WATER', 'Noise - Vehicle',
       'PLUMBING', 'Lost Property', 'Noise', 'Traffic Signal Condition',
       'WATER LEAK', 'PAINT/PLASTER', 'Consumer Complaint',
       'Curb Condition', 'Taxi Complaint', 'Elevator', 'Noise - Park',
       'Abandoned Vehicle', 'Rodent', 'Food Establishment',
       'Emergency Response Team (ERT)', 'General Construction/Plumbing',
       'Non-Emergency Police Matter', 'UNSANITARY CONDITION',
       'FLOORING/STAIRS', 'ELECTRIC', 'DOOR/WINDOW', 'SAFETY', 'GENERAL',
       'Sewer', 'Air Quality', 'Dirty Conditions', 'Pet Shop',
       'Maintenance or Facility', 'APPLIANCE',
       'Unsanitary Animal Pvt Property', 'Other Enforcement',
       'Sanitation Condition', 'Indoor Air Quality', 'ELEVATOR',
       'LinkNYC', 'Traffic', 'Pl

In [54]:
# The values in the 'agency' column also have inconsistent formatting
df.agency.unique()

array(['DEP', 'NYPD', 'DOT', 'DPR', 'HPD', 'TLC', 'DCA', 'DOB', 'DOHMH',
       'DSNY', 'DOITT', 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT',
       'EDC', 'DOE', 'DHS'], dtype=object)

In [55]:
# Among noise-related complaints, only one does not specify the noise source:
noise_columns = []

for i in df.complaint_type:
    if i.startswith('Noise') and i not in noise_columns:
        noise_columns.append(i)
noise_columns

['Noise - Street/Sidewalk',
 'Noise - Residential',
 'Noise - Commercial',
 'Noise - Vehicle',
 'Noise',
 'Noise - Park',
 'Noise - Helicopter',
 'Noise - House of Worship']

In [56]:
# From the result above, we found that there's one generic 'Noise'
# Below, I will convert the "Noise' only value to 'Noise - Unspecified' to match other similar complaint types

def update_noise(complaint):
    if complaint == 'Noise':
        return "Noise - Unspecified"
    else:
        return complaint
df.complaint_type = df.complaint_type.map(
    lambda complaint: update_noise(complaint))

### Convert Date value to string values
The 'created_date' column currently contains string values. Next, we will convert those 'created_date' string values to a datetime object.

In [57]:
# Identify the 'created_date' value
type(df.created_date[1])

pandas._libs.tslibs.timestamps.Timestamp

In [58]:
# Convert each string value to a datetime object using 'pandas.to_datetime()'
df['created_date'] = pd.to_datetime(df['created_date'],
                                   format = '%Y-%m-%dT%H:%M:%S.%f')

In [59]:
# Confirm the new data type for 'created-data'
type (df['created_date'][1])

pandas._libs.tslibs.timestamps.Timestamp

### Create columns with datetime format: day, month, day of week, and hour
Using the datetime format, we can now create new columns with the day, month, day of week, and hour of each call. This will be useful for exploring the independent relationship of each variable to other characteristics of the calls.

In [60]:
df ['day'] = [str(i.date()) for i in df.created_date]
df['month'] = [int(i.month) for i in df.created_date]
df['day_of_week'] = [int(i.weekday()) for i in df.created_date]
df['hour'] = [int(i.hour) for i in df.created_date]

In [61]:
#reorder df from least-most recent calls
df = df.reindex(index=df.index[::-1])
print(df)

              created_date           complaint_type agency community_board  \
830906 2021-10-11 23:59:55          Illegal Parking   NYPD        10 BRONX   
830905 2021-10-11 23:59:43          Noise - Vehicle   NYPD        05 BRONX   
830904 2021-10-11 23:59:27         Blocked Driveway   NYPD        09 BRONX   
830903 2021-10-11 23:59:24       Noise - Commercial   NYPD     04 BROOKLYN   
830902 2021-10-11 23:59:21          Noise - Vehicle   NYPD    10 MANHATTAN   
...                    ...                      ...    ...             ...   
4      2021-07-01 00:00:41  Noise - Street/Sidewalk   NYPD     14 BROOKLYN   
3      2021-07-01 00:00:30          Illegal Parking   NYPD     10 BROOKLYN   
2      2021-07-01 00:00:05          Illegal Parking   NYPD     14 BROOKLYN   
1      2021-07-01 00:00:00             Water System    DEP     08 BROOKLYN   
0      2021-07-01 00:00:00             Water System    DEP        07 BRONX   

         latitude  longitude                     descriptor    

In [62]:
#preview sample of unique descriptors with non-letters removed
df.descriptor.unique()[0:20]

array(['Commercial Overnight Parking', 'Car/Truck Music', 'No Access',
       'Loud Music/Party', 'Car/Truck Horn', 'Blocked Hydrant', 'Vendor',
       'Banging/Pounding', 'Noise: Construction Before/After Hours (NM1)',
       'Rough, Pitted or Cracked Roads', 'Loud Television',
       'Lamppost Knocked Down', 'MOLD', 'Broken Sidewalk', 'Loud Talking',
       'Street Light Out', 'Engine Idling', 'Blocked Sidewalk',
       'Noise: air condition/ventilation equipment (NV1)',
       'Signs of Rodents'], dtype=object)

## Merging the Community District Indicators Data with 311 Service Requests Data
This part will be started by taking a look into the community district indicators data. Next, we will be merging other relevant information obtained of Community District Indicators data to our 311 Service Request data.

In [63]:
path = '../nfh2118/Assignment 02/Queens-4-indicators.csv'
indicators_df = pd.read_csv(path)

indicators_df.head()

,the_geom,cartodb_id,the_geom_webmercator,acres,acs_tooltip,acs_tooltip_2,acs_tooltip_3,area_sqmi,borocd,cb_email,...,under18_rate,under18_rate_boro,under18_rate_nyc,unemployment_boro,unemployment,unemployment_nyc,v_pluto,v_acs,v_facdb,v_crime
0,NaN,1,NaN,976.3,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.5,101,man01@cb.nyc.gov,...,12.7,14.4,20.9,3.8,2.8,4.4,20v4,Y2014-2018,6/24/20,2019
1,NaN,2,NaN,865.9,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.4,102,bgormley@cb.nyc.gov,...,12.7,14.4,20.9,3.8,2.8,4.4,20v4,Y2014-2018,6/24/20,2019
2,NaN,3,NaN,1076.9,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.7,103,mn03@cb.nyc.gov,...,11.4,14.4,20.9,3.8,3.6,4.4,20v4,Y2014-2018,6/24/20,2019
3,NaN,4,NaN,1131.6,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.8,104,jbodine@cb.nyc.gov,...,8.2,14.4,20.9,3.8,3.5,4.4,20v4,Y2014-2018,6/24/20,2019
4,NaN,5,NaN,1005.4,American Community Survey 2014-2018 5-Year Est...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,1.6,105,office@cb5.org,...,8.2,14.4,20.9,3.8,3.5,4.4,20v4,Y2014-2018,6/24/20,2019


In [64]:
indicators_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Columns: 188 entries, the_geom to v_crime
dtypes: bool(1), float64(121), int64(51), object(15)
memory usage: 86.4+ KB


Next, we will narrow down our selection to variables related to demographic, social and economic indicators. Most of the indicators column names are pretty mush self-explanatory. However, to better understans their meanings, the full list of column descriptions can be accessed through the community district profile website in the 'Indicators Data Dictionary' file.

In [65]:
indicators_df = indicators_df[[
'cd_short_title', 
'area_sqmi',
'count_hosp_clinic',
'count_libraries',
'count_parks',
'count_public_schools',
'crime_per_1000',
'female_15_19',
'female_20_24',
'female_25_29',
'female_30_34',
'female_35_39',
'female_40_44',
'female_45_49',
'female_50_54',
'female_55_59',
'female_60_64',
'male_10_14',
'male_15_19',
'male_20_24',
'male_25_29',
'male_30_34',
'male_35_39',
'male_40_44',
'male_45_49',
'male_50_54',
'male_55_59',
'male_60_64',
'under18_rate',
'over65_rate',
'lots_total',
'lots_vacant',
'pct_white_nh',
'pct_black_nh',
'pct_asian_nh',
'pct_other_nh',
'pct_hispanic',
'mean_commute',
'pop_change_00_10',
'pct_clean_strts',
'unemployment',
'poverty_rate',
'lep_rate_boro',
'pct_hh_rent_burd'
]]

In [66]:
#Summary statistics for each of the numeric variables in the indicators dataset
indicators_df.describe().loc[['mean','std','min','max']]

,area_sqmi,count_hosp_clinic,count_libraries,count_parks,count_public_schools,crime_per_1000,female_15_19,female_20_24,female_25_29,female_30_34,...,pct_asian_nh,pct_other_nh,pct_hispanic,mean_commute,pop_change_00_10,pct_clean_strts,unemployment,poverty_rate,lep_rate_boro,pct_hh_rent_burd
mean,4.613559,22.898305,3.677966,6.915254,25.694915,10.205085,2.720339,3.540678,4.883051,4.398305,...,13.088136,2.927119,29.898305,41.238983,0.038475,94.064407,4.513559,19.940678,23.049153,44.723729
std,4.295164,13.101323,1.665478,4.035567,8.987068,4.627594,0.683243,0.844887,1.364848,1.113629,...,12.294884,2.074129,20.544053,6.309572,0.111801,3.076146,1.465518,6.824982,5.305637,7.550322
min,1.400000,1.000000,1.000000,1.000000,8.000000,2.600000,1.100000,1.800000,2.900000,3.000000,...,0.700000,1.200000,6.500000,25.400000,-0.100000,85.100000,2.000000,7.200000,10.900000,28.800000
max,21.500000,54.000000,9.000000,20.000000,47.000000,29.400000,4.200000,6.900000,8.400000,6.900000,...,52.700000,14.900000,71.500000,51.100000,0.770000,99.100000,7.800000,35.600000,28.800000,60.300000


We can see that the average size of a community district is 4.6 miles, and that the average percentage of men and women in their 20s and 30s are higher than percentages for other age groups. On average, white residents make up about a third of the community districts' population, followed by hispanic, black, and asian residents. The average poverty rate is 19%, the average unemployment rate is 4.5%, and the average crime rate (for 7 major felonies) per 1000 residents is about 10.

We'll format the community district column in this dataset such that it can be merged with the community board column in the 311 dataset. In NYC, community districts and their corresponding boards share the same number and borough. According to the webpage for Queens Community Board 3, "each community district has a Community Board to oversee the delivery of city services and facilitate local governance." Currently, the values for community boards in df and community districts in ind_df are formatted differently:

In [67]:
df.community_board[0:10]

830906        10 BRONX
830905        05 BRONX
830904        09 BRONX
830903     04 BROOKLYN
830902    10 MANHATTAN
830901     10 BROOKLYN
830900       07 QUEENS
830899     03 BROOKLYN
830898       04 QUEENS
830897       05 QUEENS
Name: community_board, dtype: object

In [68]:
indicators_df.cd_short_title[0:10]

0     Manhattan CD 1
1     Manhattan CD 2
2     Manhattan CD 3
3     Manhattan CD 4
4     Manhattan CD 5
5     Manhattan CD 6
6     Manhattan CD 7
7     Manhattan CD 8
8     Manhattan CD 9
9    Manhattan CD 10
Name: cd_short_title, dtype: object

In [69]:
#format community district string values so that they match community board vals 

pd.options.mode.chained_assignment = None 

for row in indicators_df.index:
    upper = indicators_df.at[row, "cd_short_title"].upper().replace(' CD','')
    num_first = "{}".format(upper[-2:] + ' ' + upper[:-2]).strip()
    indicators_df.cd_short_title.loc[row] = num_first

The below function removes all 0's from the beginning of single-digit community board numbers in the 311 dataset. The subsequent list comprehension uses the function to update each value in the community board column.

In [70]:
def format_cb(cb_string):

  """Takes in a string value for a community board in the 311 dataframe. If the 
     string starts with '0', updates the string to contain every letter after 0."""  
  
  if cb_string.startswith('0'):
        return cb_string[1:]
  else:
        return cb_string

In [71]:
df.community_board = [format_cb(i) for i in df.community_board]

The community board column contains more unique values than the community district column, as shown below.

In [72]:
len(df.community_board.unique())

77

In [73]:
len(indicators_df.cd_short_title.unique())

59

We'll remove rows that contain unmatching value through these steps:
1) Labelling the non-matches data; then
2) Overwriting the 311 dataset to exclude those non-matches data.

In [76]:
# Defining the non-matches data
def label_non_match(cb):

  if cb in indicators_df.cd_short_title.unique():
        return cb 
  
  else:
        return 'Non-match'

In [77]:
df.community_board = df.community_board.map(lambda cb: label_non_match(cb))

In [78]:
# Exclude the non-matches data
df = df[df.community_board != 'Non-match']

Now that all of the community district/board values have a match in both columns, we can create a column with a matching title in the indicators dataframe and perform an inner join with the 311 dataframe.

In [79]:
# Create new column with matching name to join data, and drop duplicate column
indicators_df['community_board'] = indicators_df.cd_short_title
df = pd.merge(df, indicators_df, how='inner')

df.drop("cd_short_title", axis=1, inplace=True)

## Exporting the Data

Then, I exported the dataframes that will be used for visualization.

In [80]:
df.to_csv('../nfh2118/Assignment 02/community_board_311.csv', header=True)